# Integrazione dei dataset Twitch-Giantbomb e IGDB

In [1]:
import pandas as pd
import json 
import fuzzy_pandas as fpd
import numpy

In [2]:
Twitch_Giantbomb = pd.read_json(r"C:\Users\39331\Desktop\Twitch_Giantbomb.json")

In [3]:
IGDB = pd.read_json(r"C:\Users\39331\Desktop\IGDB_BIS.json")

In [4]:
Twitch_Giantbomb = Twitch_Giantbomb.rename(columns = {'name':'name_g', 'similar_games':'similar_games_g'})

In [5]:
Twitch_Giantbomb.head(1)

,giantbomb_id,views,name_g,original_release_date,platforms,characters,developers,genres,similar_games_g,themes,dlcs,norm_name
0,37030,"[{'timestamp': '2020-07-02 08:30:53', 'visuals...",Fortnite,2017-07-25,"[{'name': 'Mac'}, {'name': 'PC'}, {'name': 'iP...","[{'name': 'Batman'}, {'name': 'Catwoman'}, {'n...","[{'date_founded': '1991-01-01', 'developed_gam...",[{'name': 'Shooter'}],"[{'name': 'CastleMiner'}, {'name': 'Minecraft'...","[{'name': 'Horror'}, {'name': 'Fantasy'}, {'na...",None,fortnite


In [6]:
IGDB.head(1)

,name,age_ratings,game_modes,normalized_name,release_dates,similar_games,time_to_beat
0,Rune Factory 4 Special,[E10],[{'name': 'Single player'}],runefactory4special,2019,"[{'name': 'God Eater 3'}, {'name': 'Shadows: A...",None


Si inzia l'integrazione facendo un inner join tradizionale, dove la chiave è il nome normalizzato. 

In [7]:
Primo_step = fpd.fuzzy_merge(Twitch_Giantbomb, IGDB, 
                            left_on = "norm_name", right_on = "normalized_name", join = "inner")

In [8]:
def calc_diff(row):
    try:
        dif = numpy.abs(row['release_dates'] - pd.to_datetime(row['original_release_date']).year)
    except:
        dif = 0
    return(dif)

Per precauzione non si ritiene però sufficiente considerare due giochi con lo stesso nome normalizzato lo stesso gioco. Si aggiunge come condizione che i giochi con lo stesso nome normalizzato devono avere una data di pubblicazione indicata su Giantbomb e IGDB che differisce al massimo di 3 anni. 

In [9]:
Primo_step['Diff'] = Primo_step[['release_dates', 'original_release_date']].apply(calc_diff, axis = 1)

Aggiunta la colonna che contiene la differenza tra gli anni, si fa una semplice query.

In [10]:
Primo_step = Primo_step[Primo_step['Diff'] <= 3]

In [11]:
Primo_step.shape

(828, 20)

Per ridurre seppur di poco i tempi di esecuzione si rimuovono dai dataset i giochi già inegrati. Ha senso in quanto un gioco già integrato non ha senso che possa integrarsi nuovamente nella prossima fase.

In [12]:
Da_Integrare_TG = Twitch_Giantbomb[Twitch_Giantbomb['name_g'].isin(Primo_step['name_g'].unique()) == False]

In [13]:
Da_Integarre_IGDB = IGDB[IGDB['name'].isin(Primo_step['name'].unique()) == False]

La seconda fase consiste nel cercare di integrare i giochi rimanenti tramite un inner join dove la chiave è il nome (non normalizzato del gioco) e aggiungendo come condizione che due records si integrano solo se la similiarità tra le stringhe del nome è almeno pari al 60%. La similiarità è calcolata tramite la edit distance (distanza di levensthein).

In [14]:
Secondo_step = fpd.fuzzy_merge(Da_Integrare_TG, Da_Integarre_IGDB, 
                            left_on = "name_g", right_on = "name", join = "inner", 
                               method = "levenshtein", threshold = 0.6, ignore_nonlatin = False)

In [15]:
Secondo_step['Diff'] = Secondo_step[['release_dates', 'original_release_date']].apply(calc_diff, axis = 1)

In [16]:
Secondo_step = Secondo_step[Secondo_step['Diff'] <= 3]

Per lo stesso motivo di prima si integrano poi i giochi con una differrnza di 3 anni al massimo nella loro pubblicazione fra i due dataset.

In [17]:
Secondo_step = Secondo_step.drop_duplicates(subset = ['name_g'], keep = "first")

I giochi pokemon presentano dei duplicati dal lato di Giantbomb, pertanto si rimuovono tali duplicati.

In [18]:
Secondo_step = Secondo_step.reset_index()

In [19]:
Secondo_step.shape

(23, 21)

In [20]:
Integrato = pd.concat([Primo_step, Secondo_step])

C:\Users\39331\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [21]:
Integrato.shape

(851, 21)

E' stato integrato il 90% del dataset di Giantbomb.

In [22]:
Integrato.to_json(r"C:\Users\39331\Desktop\Integrato.json", orient = "records")